<a href="https://colab.research.google.com/github/Zfeng0207/FIT3199-FYP/blob/dev%2Fzfeng/NB_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import gdown

import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FIT3199-FYP/input/00_recurrent_stroke_patient.csv')

,subject_id,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,...,icd_code,icd_version,icd_title,numeric_part,Stroke_Y/N,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10003299,31931401,2183-02-26 20:26:00,97.7,63.0,15.0,97.0,146.0,65.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
1,10003299,31931401,2183-02-26 21:43:00,NaN,51.0,16.0,97.0,131.0,42.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
2,10003299,31931401,2183-02-26 22:03:00,NaN,57.0,19.0,100.0,142.0,54.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
3,10003299,31931401,2183-02-27 02:20:00,NaN,54.0,18.0,97.0,129.0,56.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
4,10003299,31931401,2183-02-27 06:26:00,98.7,61.0,15.0,98.0,123.0,53.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67962,19992885,35746474,2152-07-08 01:28:00,98.0,74.0,18.0,99.0,118.0,55.0,NaN,...,4423,9,LOWER EXTREMITY ANEURYSM,4423,0,M,50,2152,2014 - 2016,2152-07-24
67963,19992885,35746474,2152-07-08 01:28:00,98.0,74.0,18.0,99.0,118.0,55.0,NaN,...,E8788,9,ABN REACT-SURG PROC NEC,8788,0,M,50,2152,2014 - 2016,2152-07-24
67964,19992885,35746474,2152-07-08 03:34:00,97.8,78.0,16.0,96.0,125.0,74.0,NaN,...,9972,9,SURG COMP-PERI VASC SYST,9972,0,M,50,2152,2014 - 2016,2152-07-24
67965,19992885,35746474,2152-07-08 03:34:00,97.8,78.0,16.0,96.0,125.0,74.0,NaN,...,4423,9,LOWER EXTREMITY ANEURYSM,4423,0,M,50,2152,2014 - 2016,2152-07-24


In [7]:
print(data.columns)

Index(['subject_id', 'stay_id', 'charttime', 'temperature', 'heartrate',
       'resprate', 'o2sat', 'sbp', 'dbp', 'rhythm', 'pain', 'seq_num',
       'icd_code', 'icd_version', 'icd_title', 'numeric_part', 'Stroke_Y/N',
       'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod'],
      dtype='object')


In [9]:
processed_df = data.fillna({
    'subject_id': 0,  # Integer columns -> 0
    'stay_id_x': 0.0,  # Float columns -> 0.0
    'temperature': 0.0,  # Float columns -> 0.0
    'heartrate': 0.0,  # Float columns -> 0.0
    'resprate': 0.0,  # Float columns -> 0.0
    'o2sat': 0.0,  # Float columns -> 0.0
    'sbp': 0.0,  # Float columns -> 0.0
    'dbp': 0.0,  # Float columns -> 0.0
    'rhythm': 0,  # Integer columns -> 0
    'pain': 0,  # Integer columns -> 0
    'year': 0,  # Float columns -> 0.0
    'month': 0,  # Integer columns -> 0
    'day': 0,  # Integer columns -> 0
    'hour': 0,  # Integer columns -> 0
    'minute': 0,  # Integer columns -> 0
    'second': 0,  # Integer columns -> 0
    'dod_year': 0.0,  # String columns -> 'null'
    'dod_month': 0.0,  # String columns -> 'null'
    'dod_day': 0.0  # String columns -> 'null'
})

In [16]:
data.dtypes

,0
stay_id,int64
temperature,float64
heartrate,float64
resprate,float64
o2sat,float64
sbp,float64
dbp,float64
rhythm,object
pain,object
seq_num,int64


In [17]:
# Drop non-relevant columns
columns_to_drop = ["subject_id", "stay_id_x", "stay_id_y", "charttime", "dod", "icd_title","icd_code","icd_version","gender","anchor_year_group","pain",'rhythm']
data = processed_df.drop(columns=columns_to_drop, errors='ignore')

# Define target variable
target_column = "Stroke_Y/N"
X = data.drop(columns=[target_column]).values
y = data[target_column].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Split dataset into train, validation, and test sets
train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_data, val_data, test_data = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Train Naive Bayes Model
def train_naive_bayes(train_loader, val_loader):
    X_train, y_train = [], []
    for X_batch, y_batch in train_loader:
        X_train.append(X_batch.numpy())
        y_train.append(y_batch.numpy())
    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)

    model = GaussianNB()
    model.fit(X_train, y_train)

    # Validation Accuracy
    X_val, y_val = [], []
    for X_batch, y_batch in val_loader:
        X_val.append(X_batch.numpy())
        y_val.append(y_batch.numpy())
    X_val = np.vstack(X_val)
    y_val = np.hstack(y_val)

    y_val_pred = model.predict(X_val)
    val_acc = accuracy_score(y_val, y_val_pred)
    print(f'Validation Accuracy: {val_acc:.4f}')
    return model

nb_model = train_naive_bayes(train_loader, val_loader)

# Evaluate Naive Bayes Model
def evaluate_naive_bayes(model, test_loader):
    X_test, y_test = [], []
    for X_batch, y_batch in test_loader:
        X_test.append(X_batch.numpy())
        y_test.append(y_batch.numpy())
    X_test = np.vstack(X_test)
    y_test = np.hstack(y_test)

    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    print(f'Test Accuracy: {test_acc:.4f}')

evaluate_naive_bayes(nb_model, test_loader)


Validation Accuracy: 0.3690
Test Accuracy: 0.3721


In [18]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluate Naive Bayes Model
def evaluate_naive_bayes(model, test_loader):
    X_test, y_test = [], []
    for X_batch, y_batch in test_loader:
        X_test.append(X_batch.numpy())
        y_test.append(y_batch.numpy())
    X_test = np.vstack(X_test)
    y_test = np.hstack(y_test)

    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    print(f'Test Accuracy: {test_acc:.4f}')
    return y_test, y_test_pred

y_test, y_test_pred = evaluate_naive_bayes(nb_model, test_loader)

# Calculate additional performance metrics
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

Test Accuracy: 0.3721
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.27      0.43      8771
         1.0       0.18      0.98      0.30      1425

    accuracy                           0.37     10196
   macro avg       0.58      0.63      0.37     10196
weighted avg       0.87      0.37      0.41     10196

Confusion Matrix:
[[2402 6369]
 [  33 1392]]


In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np

# Number of folds
k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Store performance metrics
accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

# Convert data from DataLoader to NumPy arrays
X_np, y_np = [], []
for X_batch, y_batch in test_loader:
    X_np.append(X_batch.numpy())
    y_np.append(y_batch.numpy())
X_np = np.vstack(X_np)
y_np = np.hstack(y_np)

# Perform K-Fold Cross-Validation
for train_index, test_index in kf.split(X_np, y_np):
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]

    # Train Naïve Bayes model
    nb_model.fit(X_train, y_train)

    # Make predictions
    y_pred = nb_model.predict(X_test)

    # Compute metrics
    accuracy_list.append(accuracy_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    f1_list.append(f1_score(y_test, y_pred))

# Print average results
print(f"Naïve Bayes Model - {k}-Fold Cross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}")
print(f"Average Precision: {np.mean(precision_list):.4f} ± {np.std(precision_list):.4f}")
print(f"Average Recall: {np.mean(recall_list):.4f} ± {np.std(recall_list):.4f}")
print(f"Average F1-score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}")

from sklearn.metrics import confusion_matrix

# Display Final Evaluation Metrics
print("\n--- Naïve Bayes Model - Final K-Fold Performance ---")
print(f"Mean Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}")
print(f"Mean Precision: {np.mean(precision_list):.4f} ± {np.std(precision_list):.4f}")
print(f"Mean Recall: {np.mean(recall_list):.4f} ± {np.std(recall_list):.4f}")
print(f"Mean F1-score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}")

# Get final predictions on the last test set for confusion matrix
y_pred_final = nb_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_final)

# Display Confusion Matrix
print("\nConfusion Matrix:")
print(cm)



Naïve Bayes Model - 10-Fold Cross-Validation Results:
Average Accuracy: 0.8594 ± 0.0020
Average Precision: 0.0885 ± 0.1788
Average Recall: 0.0098 ± 0.0217
Average F1-score: 0.0171 ± 0.0373

--- Naïve Bayes Model - Final K-Fold Performance ---
Mean Accuracy: 0.8594 ± 0.0020
Mean Precision: 0.0885 ± 0.1788
Mean Recall: 0.0098 ± 0.0217
Mean F1-score: 0.0171 ± 0.0373

Confusion Matrix:
[[876   1]
 [142   0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.p

In [20]:
from sklearn.metrics import confusion_matrix

# Initialize lists to collect all predictions and true labels across folds
all_y_true = []
all_y_pred = []

# Perform K-Fold Cross-Validation and collect results
for train_index, test_index in kf.split(X_np, y_np):
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]

    # Train Naïve Bayes model
    nb_model.fit(X_train, y_train)

    # Make predictions
    y_pred = nb_model.predict(X_test)

    # Store predictions and true values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

# Compute overall confusion matrix
overall_cm = confusion_matrix(all_y_true, all_y_pred)

# Display Confusion Matrix
print("\nOverall Confusion Matrix Across All K-Folds:")
print(overall_cm)



Overall Confusion Matrix Across All K-Folds:
[[8748   23]
 [1411   14]]


In [ ]:
data

In [ ]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Define target variable
target_column = "Stroke_Y/N"
X = data.drop(columns=[target_column]).values
y = data[target_column].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Split dataset into train, validation, and test sets
train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_data, val_data, test_data = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Train SVM Model
def train_svm(train_loader, val_loader):
    X_train, y_train = [], []
    for X_batch, y_batch in train_loader:
        X_train.append(X_batch.numpy())
        y_train.append(y_batch.numpy())
    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)

    model = SVC(kernel='rbf', probability=True)
    model.fit(X_train, y_train)

    # Validation Accuracy
    X_val, y_val = [], []
    for X_batch, y_batch in val_loader:
        X_val.append(X_batch.numpy())
        y_val.append(y_batch.numpy())
    X_val = np.vstack(X_val)
    y_val = np.hstack(y_val)

    y_val_pred = model.predict(X_val)
    val_acc = accuracy_score(y_val, y_val_pred)
    print(f'Validation Accuracy: {val_acc:.4f}')
    return model

svm_model = train_svm(train_loader, val_loader)

# Evaluate SVM Model
def evaluate_svm(model, test_loader):
    X_test, y_test = [], []
    for X_batch, y_batch in test_loader:
        X_test.append(X_batch.numpy())
        y_test.append(y_batch.numpy())
    X_test = np.vstack(X_test)
    y_test = np.hstack(y_test)

    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    print(f'Test Accuracy: {test_acc:.4f}')
    return y_test, y_test_pred

y_test, y_test_pred = evaluate_svm(svm_model, test_loader)

# Calculate additional performance metrics
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Define number of folds
K = 5
kf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

# Initialize lists to store performance metrics across folds
accuracy_list, precision_list, recall_list, f1_list = [], [], [], []
all_y_true, all_y_pred = [], []

# Perform K-Fold Cross-Validation
for train_index, test_index in kf.split(X_np, y_np):
    # Split data into training and testing for this fold
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]

    # Train SVM model
    svm_model = SVC(kernel='rbf', C=1.0, probability=True)  # RBF kernel, tune hyperparameters as needed
    svm_model.fit(X_train, y_train)

    # Make predictions
    y_pred = svm_model.predict(X_test)

    # Store true labels and predictions for confusion matrix
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Compute performance metrics for this fold
    accuracy_list.append(accuracy_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    f1_list.append(f1_score(y_test, y_pred))

# Compute overall confusion matrix
overall_cm = confusion_matrix(all_y_true, all_y_pred)

# Display Final Evaluation Metrics
print("\n--- SVM Model - Final K-Fold Performance ---")
print(f"Mean Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}")
print(f"Mean Precision: {np.mean(precision_list):.4f} ± {np.std(precision_list):.4f}")
print(f"Mean Recall: {np.mean(recall_list):.4f} ± {np.std(recall_list):.4f}")
print(f"Mean F1-score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}")

# Display Confusion Matrix
print("\nOverall Confusion Matrix Across All K-Folds:")
print(overall_cm)
